In [1]:
!pip install jsonlines

In [2]:
!pip install opendatasets

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
import jsonlines
import pandas as pd

In [ ]:
! pip install kaggle



In [7]:
! mkdir ~/.kaggle



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
! cp kaggle.json ~/.kaggle/



In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [25]:
!cd /content

In [11]:
! kaggle datasets download ahmadsameh7810/aic-datasets

 62% 8.00M/12.9M [00:00<00:00, 31.7MB/s]
100% 12.9M/12.9M [00:00<00:00, 46.4MB/s]


In [12]:
! unzip /content/aic-datasets.zip

Archive:  /content/aic-datasets.zip
  inflating: test.jsonl              
  inflating: train.jsonl             
  inflating: validation.jsonl        


In [13]:
def read_jsonl_to_dataframe(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            data.append(line)
    df = pd.DataFrame(data)
    return df

In [14]:
train = read_jsonl_to_dataframe('train.jsonl')
validation = read_jsonl_to_dataframe('validation.jsonl')
test = read_jsonl_to_dataframe('test.jsonl')

In [ ]:
validation_data = read_jsonl_to_dataframe('validation_data.jsonl')

In [ ]:
validation_data

,example_id,paragraph
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...
...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...


In [32]:
!pip install transformers

In [33]:
from transformers import AutoTokenizer



In [44]:
!pip install torch

In [45]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mt5_config = AutoConfig.from_pretrained("google/mt5-large",max_length=515,length_penalty=0.6,
  no_repeat_ngram_size=2,
  num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-large", config=mt5_config)
         .to(device))
tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")

NameError: ignored

In [38]:
from datasets import Dataset
train = Dataset.from_dict(train)
test = Dataset.from_dict(test)
validation = Dataset.from_dict(validation)

In [ ]:
validation_data = Dataset.from_dict(validation_data)

In [ ]:
validation_data

Dataset({
    features: ['example_id', 'paragraph'],
    num_rows: 272
})

In [39]:
def tokenize_sample_data(data):
  input_feature = tokenizer(data["predicted"], truncation=True, max_length=515)
  label = tokenizer(data["summary"], truncation=True, max_length=236)
  return {
    "input_ids": input_feature["input_ids"],
    "attention_mask": input_feature["attention_mask"],
    "labels": label["input_ids"],
  }

In [40]:
tokenized_train = train.map(tokenize_sample_data, batched=True, batch_size=128,remove_columns=["predicted", "summary"])
tokenized_test = test.map(tokenize_sample_data, batched=True, batch_size=128,remove_columns=["predicted", "summary"])
tokenized_validation = validation.map(tokenize_sample_data, batched=True, batch_size=128,remove_columns=["predicted", "summary"])


Map:   0%|          | 0/37519 [00:00<?, ? examples/s]

NameError: ignored

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 37519
})

C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmad\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  return_tensors="pt")

In [ ]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = tokenizer(arg)
  return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
  text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
  sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [ ]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_test.with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break

metrics_func([preds, labels])

{'rouge1': 0.024487520843657518,
 'rouge2': 0.012395768208847932,
 'rougeL': 0.02448752084365752,
 'rougeLsum': 0.024487520843657518}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
  output_dir = "output",
  log_level = "error",
  num_train_epochs = 1,
  learning_rate = 5e-4,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size = 2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 100,
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False
)

from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  # compute_metrics = metrics_func,
  train_dataset = tokenized_train,
  eval_dataset = tokenized_validation,
  tokenizer = tokenizer,
)


In [ ]:
trainer.train()

  0%|          | 0/11720 [00:26<?, ?it/s]


{'loss': 14.3585, 'learning_rate': 5.555555555555555e-05, 'epoch': 0.01}
{'loss': 10.8539, 'learning_rate': 0.0001111111111111111, 'epoch': 0.02}
{'loss': 10.1153, 'learning_rate': 0.00016666666666666666, 'epoch': 0.03}
{'loss': 9.7457, 'learning_rate': 0.0002222222222222222, 'epoch': 0.03}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\ahmad\AppData\Local\Temp\ipykernel_9712\4032920361.py:1 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\ahmad\\AppData\\Local\\Temp\\ipykernel_9712\\4032920361.py'                          │
│                                                                                                  │
│ C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCac │
│ he\local-packages\Python39\site-packages\transformers\trainer.py:1645 in train                   │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCac │
│ he\local-packages\Python39\site-packages\transformers\trainer.py:1938 in _inner_training_loop    │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCac │
│ he\local-packages\Python39\site-packages\transformers\trainer.py:2770 in training_step           │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundat